In [ ]:
IMG_SIZE = 80

## Creating the Convolutional Neural Network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(IMG_SIZE,IMG_SIZE).view(-1,1,IMG_SIZE,IMG_SIZE)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))        
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        #print(x[0].shape)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
net = Net()

### Read the data from file

In [3]:
import numpy as np
training_data = np.load('training_data.npy', allow_pickle=True)
print(len(training_data))

8736


### Split the data up into trainig and test data

In [4]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1, IMG_SIZE, IMG_SIZE)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(str(val_size) + ' test samples')

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

873 test samples


### Train the model (old way)

### Test the model for accuracy by percent and with Heidke Skill Score (old way)

### Calculate loss and accuracy for loging

In [5]:
from stats import heidke_skill_score, f1_score, skill_score

def fwd_pass(X, y, train=False):
    if train:
        net.zero_grad()
    outputs = net(X)
    matches = [torch.argmax(i) == torch.argmax(j) for i, j in zip(outputs, y)]
    #acc = matches.count(True)/len(matches)
    #acc = (((2*len(matches))-matches.count(False))/matches.count(False))
    #acc = heidke_skill_score(y, matches)
    #acc = f1_score(y, matches)
    acc = skill_score(matches)
    loss = loss_function(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
    return acc, loss

### Test function for loging

In [6]:
def test(size=32):
    random_start = np.random.randint(len(test_X)-size)
    X, y = test_X[random_start:random_start+size], test_y[random_start:random_start+size]
    val_acc, val_loss = fwd_pass(X.view(-1,1,IMG_SIZE,IMG_SIZE), y)
    return val_acc, val_loss

### Train function with loging

In [7]:
import time
from tqdm import tqdm
from datetime import datetime

MODEL_NAME = f"model-{int(time.time())}"

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

print(MODEL_NAME)

def train():
    start = datetime.now()
    BATCH_SIZE = 50
    EPOCHS = 5
    with open("model.log", "a") as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
                batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,IMG_SIZE,IMG_SIZE)
                batch_y = train_y[i:i+BATCH_SIZE]
                
                acc, loss = fwd_pass(batch_X, batch_y, train=True)
                
                number_of_datapoints_for_one_log = 100
                if i % number_of_datapoints_for_one_log == 0:
                    val_acc, val_loss = test(size=10)
                    f.write(f"{MODEL_NAME}, {round(time.time(), 3)}, {round(float(acc), 2)}, {round(float(loss), 4)}, {round(float(val_acc), 2)}, {round(float(val_loss), 4)}, {epoch}\n")
    end = datetime.now()
    return BATCH_SIZE, EPOCHS, int(time.time()), str(end-start)

batch_size, epochs, timestamp, timedelta = train()

print('The training took: {}'.format(timedelta))

torch.save(net, 'model-{}-{}-{}.pt'.format(batch_size, epochs, timestamp))

  0%|          | 0/158 [00:00<?, ?it/s]

model-1587384020


100%|██████████| 158/158 [00:44<00:00,  3.52it/s]

The training took: 0:03:40.297944



/home/pirate/Desktop/solarflare-catcher/sdobenchmark/sdobenchmarkvenv/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Graph function

In [12]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style
from datetime import datetime

style.use("ggplot")

def create_acc_loss_graph(model_name):
    contents = open("model.log", "r").read().split('\n')
    
    times = []
    accuracies = []
    losses = []

    val_accs = []
    val_losses = []
    
    epochs = []
    epochs_times = []
    
    for c in contents:
        if model_name in c:
            name, timestamp, acc, loss, val_acc, val_loss, epoch = c.split(",")

            times.append(float(timestamp))
            accuracies.append(float(acc))
            losses.append(float(loss))

            val_accs.append(float(val_acc))
            val_losses.append(float(val_loss))
                        
            if len(epochs) == 0:
                epochs.append(float(epoch))
                epochs_times.append(float(timestamp))
            else:
                if float(epoch) != epochs[-1]:
                    epochs.append(float(epoch))
                    epochs_times.append(float(timestamp))
    
    print(epochs)
    print(epochs_times)

    fig = plt.figure("SDOBenchmark-{}".format(str(datetime.now())))

    ax1 = plt.subplot2grid((2,1), (0,0))
    ax2 = plt.subplot2grid((2,1), (1,0), sharex=ax1)

    ax1.plot(times, accuracies, label="acc (training data)")
    ax1.plot(times, val_accs, label="val_acc (test data)")
    for e in epochs_times:
       ax1.axvline(e, color="black")
    ax1.legend(loc=2)
     
    ax2.plot(times, losses, label="loss (training data)")
    ax2.plot(times, val_losses, label="val_loss (test data)")
    for e in epochs_times:
       ax2.axvline(e, color="black")
    ax2.legend(loc=2)
    

    plt.show()
    

create_acc_loss_graph(MODEL_NAME)

[0.0, 1.0, 2.0, 3.0, 4.0]
[1587384021.321, 1587384065.039, 1587384111.683, 1587384155.269, 1587384196.643]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
correct = 0
wrong = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1,1,IMG_SIZE,IMG_SIZE))[0]        
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        else:
            wrong += 1
print("Accuracy:", round(correct/(correct + wrong),3))

100%|██████████| 873/873 [00:03<00:00, 273.39it/s]

Accuracy: 0.915


In [10]:
create_acc_loss_graph

<function __main__.create_acc_loss_graph(model_name)>

In [11]:
create_acc_loss_graph("model-1586215754")

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0]
[1586215755.543, 1586215761.586, 1586215768.159, 1586215776.389, 1586215784.971, 1586215792.092, 1586215800.77, 1586215808.583, 1586215815.858, 1586215822.483, 1586215829.663, 1586215835.992, 1586215842.491, 1586215849.177, 1586215854.972, 1586215860.702, 1586215866.4, 1586215872.321, 1586215878.149, 1586215883.956, 1586215889.713, 1586215895.637, 1586215901.393, 1586215907.554, 1586215913.296, 1586215918.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …